In [1]:
from chat.tests.data.mega_corp_alpha_manifesto import manifesto
from chat.services import recursive_splitter, simple_splitter, spacy_splitter

In [2]:
manifesto

"\nThese are the 18 principles of the MegaCorpAlpha Way:\n\n1.  We expect all our leaders to embody the MegaCorpAlpha Way philosophy and mentor our team members in its tools and methodologies.\n2.  Our commitments are always backed by realistic and actionable plans.\n3.  Our choices and actions are directed by our overarching company values.\n4.  We approach management with foresight and longevity in mind.\n5.  Every MegaCorpAlpha team member is encouraged to integrate the MegaCorpAlpha Way into their daily activities.\n6.  Personal growth and internal promotions are pivotal in our growth strategy.\n7.  All of us play an essential role in bringing outstanding talent into MegaCorpAlpha.\n8.  We forge productive and future-forward partnerships, aiming for Ten(d) to Zero discrepancies.\n9.  We prioritize firsthand experiences, ensuring we truly grasp ongoing situations.\n10. Team members are empowered to address and resolve issues at their respective tiers.\n11. We emphasize incorporating

In [3]:
simple_splitter(manifesto)

['These are the 18 principles of the MegaCorpAlpha Way:\n1.  We expect all our leaders to embody the MegaCorpAlpha Way philosophy and mentor our team members in its tools and methodologies.\n2.  Our commitments are always backed by realistic and actionable plans.',
 '3.  Our choices and actions are directed by our overarching company values.\n4.  We approach management with foresight and longevity in mind.\n5.  Every MegaCorpAlpha team member is encouraged to integrate the MegaCorpAlpha Way into their daily activities.',
 '6.  Personal growth and internal promotions are pivotal in our growth strategy.\n7.  All of us play an essential role in bringing outstanding talent into MegaCorpAlpha.\n8.  We forge productive and future-forward partnerships, aiming for Ten(d) to Zero discrepancies.',
 '9.  We prioritize firsthand experiences, ensuring we truly grasp ongoing situations.\n10. Team members are empowered to address and resolve issues at their respective tiers.\n11. We emphasize incorpo

In [4]:
len(simple_splitter(manifesto))

7

In [5]:
recursive_splitter(manifesto)

['These are the 18 principles of the MegaCorpAlpha Way:',
 '1.  We expect all our leaders to embody the MegaCorpAlpha Way philosophy and mentor our team members in its tools and methodologies.\n2.  Our commitments are always backed by realistic and actionable plans.\n3.  Our choices and actions are directed by our overarching company values.',
 '4.  We approach management with foresight and longevity in mind.\n5.  Every MegaCorpAlpha team member is encouraged to integrate the MegaCorpAlpha Way into their daily activities.\n6.  Personal growth and internal promotions are pivotal in our growth strategy.',
 '7.  All of us play an essential role in bringing outstanding talent into MegaCorpAlpha.\n8.  We forge productive and future-forward partnerships, aiming for Ten(d) to Zero discrepancies.\n9.  We prioritize firsthand experiences, ensuring we truly grasp ongoing situations.',
 '10. Team members are empowered to address and resolve issues at their respective tiers.\n11. We emphasize inco

In [6]:
len(recursive_splitter(manifesto))

8

In [7]:
spacy_splitter(manifesto)

/Users/piotrgryko/code/richard/django-llama2-reactjs-chat-pdf/server/.venv/lib/python3.10/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


["These are the 18 principles of the MegaCorpAlpha Way:\n\n1.  \n\nWe expect all our leaders to embody the MegaCorpAlpha Way philosophy and mentor our team members in its tools and methodologies.\n\n\n2.  \n\nOur commitments are always backed by realistic and actionable plans.\n\n\n3.  \n\nOur choices and actions are directed by our overarching company values.\n\n\n4.  \n\nWe approach management with foresight and longevity in mind.\n\n\n5.  \n\nEvery MegaCorpAlpha team member is encouraged to integrate the MegaCorpAlpha Way into their daily activities.\n\n\n6.  \n\nPersonal growth and internal promotions are pivotal in our growth strategy.\n\n\n7.  All of us play an essential role in bringing outstanding talent into MegaCorpAlpha.\n8.  \n\nWe forge productive and future-forward partnerships, aiming for Ten(d) to Zero discrepancies.\n\n\n9.  \n\nWe prioritize firsthand experiences, ensuring we truly grasp ongoing situations.\n\n\n10.\n\nTeam members are empowered to address and resolve

In [8]:
len(spacy_splitter(manifesto))

1